In [38]:
import numpy as np

import tensorflow as tf
from checkers_load import makeBatches, load_games, dataGenerator, getData
import checkers_models
import checkers_layers
from checkers_display import BoardDisplay, SimpleSquareBoardDisplay, MoveHeatmapDisplay

import bqplot
import ipywidgets as widgets

from IPython.display import display

In [65]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.metrics import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint

from checkers_layers import *

x, y = 10, 10

# Only dense layer acc: 0.2894 (cross entropy)
def fullyConvModelSymm_test(x, y, layers, filters):
    model = Sequential()

    # Add a duplicate input for each symmetry layer.
    # This is invarient to shuffling the input.
    def lift(X):
        return tf.concat([X,X], axis=-1)
    model.add(Lambda(lift, input_shape=(x, y, 1), output_shape=(x, y, 2)))

    model.add(Conv2DSymm(filters, (3,3), activation='relu', padding='same'))
    for i in range(layers-1):
        model.add(Conv2DSymm(filters, (3,3), activation='relu', padding='same'))

    # 2 filters = 4 layer output, for each direction of thingy.
    model.add(Conv2DSymm(1, (3,3), activation=None, padding='same'))

    #model.add(Dense(128, activation='softmax'))
    model.add(Activation('softmax')) # Safe for crossentropy

    # model.add(Dense(128, activation='relu'))

    # optim = SGD()
    optim = Adam(0.0001)
    model.compile(loss='categorical_crossentropy', optimizer=optim)
    # model.compile(loss='mse', optimizer=optim, metrics=metrics)
    model.summary()

    return model

model = fullyConvModelSymm_test(x, y, 10, 10)

FLIP (None, 10, 10, 2)
trannsssssssssss (None, 10, 10, 1) (None, 10, 10, 1) (None, 10, 10, 2)
FLIP (None, 10, 10, 10)
FLIP (None, 10, 10, 20)
trannsssssssssss (None, 10, 10, 10) (None, 10, 10, 10) (None, 10, 10, 20)
FLIP (None, 10, 10, 10)
FLIP (None, 10, 10, 20)
trannsssssssssss (None, 10, 10, 10) (None, 10, 10, 10) (None, 10, 10, 20)
FLIP (None, 10, 10, 10)
FLIP (None, 10, 10, 20)
trannsssssssssss (None, 10, 10, 10) (None, 10, 10, 10) (None, 10, 10, 20)
FLIP (None, 10, 10, 10)
FLIP (None, 10, 10, 20)
trannsssssssssss (None, 10, 10, 10) (None, 10, 10, 10) (None, 10, 10, 20)
FLIP (None, 10, 10, 10)
FLIP (None, 10, 10, 20)
trannsssssssssss (None, 10, 10, 10) (None, 10, 10, 10) (None, 10, 10, 20)
FLIP (None, 10, 10, 10)
FLIP (None, 10, 10, 20)
trannsssssssssss (None, 10, 10, 10) (None, 10, 10, 10) (None, 10, 10, 20)
FLIP (None, 10, 10, 10)
FLIP (None, 10, 10, 20)
trannsssssssssss (None, 10, 10, 10) (None, 10, 10, 10) (None, 10, 10, 20)
FLIP (None, 10, 10, 10)
FLIP (None, 10, 10, 20)
tran

In [66]:
def trans(X):
    flt = X.shape[-1] // 2
    a = X[:, :, :, 0:flt]
    b = X[:, :, :, flt:]
    res = flip(np.concatenate([b,a], axis=-1))
    #print("trannsssssssssss", a.shape, b.shape, res.shape, flush=True)
    return res

# reverse x along symmetry. TODO: Check axis.
def flip(X):
    #print("FLIP", X.shape, flush=True)
    return np.flip(X, axis=-2) # When it's ?,8,4,1, it's tall and thin... Right?


In [68]:
a = np.random.rand(1,x, y,1)
b = flip(a)

a_pred = model.predict(a)
b_pred = model.predict(b)

print(a,b)

b_pred = trans(b_pred)

print(a_pred, "\n", b_pred)


print(a_pred - b_pred)

FLIP (None, 10, 10, 2)
trannsssssssssss (None, 10, 10, 1) (None, 10, 10, 1) (None, 10, 10, 2)
FLIP (None, 10, 10, 10)
FLIP (None, 10, 10, 20)
trannsssssssssss (None, 10, 10, 10) (None, 10, 10, 10) (None, 10, 10, 20)
FLIP (None, 10, 10, 10)
FLIP (None, 10, 10, 20)
trannsssssssssss (None, 10, 10, 10) (None, 10, 10, 10) (None, 10, 10, 20)
FLIP (None, 10, 10, 10)
FLIP (None, 10, 10, 20)
trannsssssssssss (None, 10, 10, 10) (None, 10, 10, 10) (None, 10, 10, 20)
FLIP (None, 10, 10, 10)
FLIP (None, 10, 10, 20)
trannsssssssssss (None, 10, 10, 10) (None, 10, 10, 10) (None, 10, 10, 20)
FLIP (None, 10, 10, 10)
FLIP (None, 10, 10, 20)
trannsssssssssss (None, 10, 10, 10) (None, 10, 10, 10) (None, 10, 10, 20)
FLIP (None, 10, 10, 10)
FLIP (None, 10, 10, 20)
trannsssssssssss (None, 10, 10, 10) (None, 10, 10, 10) (None, 10, 10, 20)
FLIP (None, 10, 10, 10)
FLIP (None, 10, 10, 20)
trannsssssssssss (None, 10, 10, 10) (None, 10, 10, 10) (None, 10, 10, 20)
FLIP (None, 10, 10, 10)
FLIP (None, 10, 10, 20)
tran

   [0.494877   0.505123  ]]]]
[[[[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0

In [69]:
# 1 liner...
a = np.random.rand(1,x, y,1)

print(model.predict(a) - trans(model.predict(flip(a))))

[[[[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]]

  [[0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0. 0.]
   [0

In [47]:
# Check I got axes right when I did the stupid code thingy.

# Create model
model = checkers_models.fullyConvModelSymm(3,3)

FLIP (None, 8, 8, 2)
trannsssssssssss (None, 8, 8, 1) (None, 8, 8, 1) (None, 8, 8, 2)
FLIP (None, 8, 8, 3)
FLIP (None, 8, 8, 6)
trannsssssssssss (None, 8, 8, 3) (None, 8, 8, 3) (None, 8, 8, 6)
FLIP (None, 8, 8, 3)
FLIP (None, 8, 8, 6)
trannsssssssssss (None, 8, 8, 3) (None, 8, 8, 3) (None, 8, 8, 6)
FLIP (None, 8, 8, 3)
FLIP (None, 8, 8, 6)
trannsssssssssss (None, 8, 8, 3) (None, 8, 8, 3) (None, 8, 8, 6)
FLIP (None, 8, 8, 2)
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_6 (Lambda)            (None, 8, 8, 2)           0         
_________________________________________________________________
conv2d_symm_27 (Conv2DSymm)  (None, 8, 8, 6)           57        
_________________________________________________________________
conv2d_symm_28 (Conv2DSymm)  (None, 8, 8, 6)           165       
_________________________________________________________________
conv2d_symm_29 (Conv2D

In [58]:
# Create UI with 3 boards: Game state, target output, network output.
figs = []

board_0 = BoardDisplay(canInteract=False)
board_1 = BoardDisplay(canInteract=False)
move_0 = MoveHeatmapDisplay()
move_1 = MoveHeatmapDisplay()
 
display(widgets.VBox([widgets.HBox([board_0.fig, move_0.fig]), widgets.HBox([board_1.fig, move_1.fig])]))

5.0
0.1
5.0
0.1


In [60]:
x = np.random.rand(1,8,4,1) - 0.5

x2 = flip(x)

#y_pred = model.predict(x)[0]

board_0.update(np.squeeze(x))
board_1.update(np.squeeze(x2))

#move_d.update(y_pred)
#pred_d.update(y_pred)
# HARD TO SPOT!! But, it's right!!

FLIP (1, 8, 4, 1)
